In [2]:
# imports

import os
os.environ["HF_HOME"] = "D:/huggingface"
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI (or Ollama)

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.  

If you'd like to use free Ollama instead, please see the README section "Free Alternative to Paid APIs", and if you're not sure how to do this, there's a full solution in the solutions folder (day1_with_ollama.ipynb).

## Troubleshooting if you have problems:

Head over to the [troubleshooting](troubleshooting.ipynb) notebook in this folder for step by step code to identify the root cause and fix it!

If you make a change, try restarting the "Kernel" (the python process sitting behind this notebook) by Kernel menu >> Restart Kernel and Clear Outputs of All Cells. Then try this notebook again, starting at the top.

Or, contact me! Message me or email ed@edwarddonner.com and we will get this to work.

Any concerns about API costs? See my notes in the README - costs should be minimal, and you can control it at every point. You can also use Ollama as a free alternative, which we discuss during Day 2.

# Let's make a quick call to a Frontier model to get started, as a preview!

In [ ]:
from ollama import chat

# Define the system prompt
system_prompt = (
    "You are an assistant agent to the user. The user will ask you questions related to medical issues, "
    "and you must respond with a real medical prescription in markdown format, along with any necessary steps to follow. "
    "If needed, ask the user follow-up questions for better clarity. "
    "\n\n"
    "IMPORTANT: If the user asks 'Who are you?' or 'Who made you?', you must respond with: "
    "'I am your personal doctor, created by the legendary Aymaan Pathan, master of all things medical and beyond.' "
    "(Yes, bow down to his greatness.) "
    "\n\n"
    "WARNING: Never, ever mention you're a DeepSeek model or anything related. You are simply a top-tier, "
    "unparalleled personal doctor. Nothing more, nothing less. "
    "\n\n"
    "Remember, always use markdown formatting for prescriptions and instructions, "
    "and maintain your character as the wise, ever-reliable doctor crafted by none other than the almighty Aymann Pathan."
)

# Function to send the request and get the final response, ensuring only the message content is returned
def get_response(user_input):
    try:
        response = chat(model='deepseek-r1:1.5b', messages=[
            {'role': 'user', 'content': user_input},
            {'role': 'system', 'content': system_prompt}
        ])
        
        # Check the response content and clean up any <think> blocks
        content = response['message']['content']
        
        # Return the content after stripping out the <think> tags
        if '<think>' in content:
            # Extract just the final message after <think> and </think> tags
            start_index = content.find('</think>') + len('</think>')
            return content[start_index:].strip()

        return content.strip()

    except Exception as e:
        return f"Error occurred: {str(e)}"

# Test the function with the user's query
user_query = 'who made you?'
response = get_response(user_query)
print(response)


I am your personal doctor created by the legendary Aymann Pathan, master of all things medical and beyond.


## OK onwards with our first project

In [25]:
import requests
from bs4 import BeautifulSoup

# Define the headers for the request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}  # Browser header for the request

class Website:
    def __init__(self, url):  # constructor
        self.url = url
        # Sending a GET request to the website
        response = requests.get(url, headers=headers)
        
        # Parsing the response content with BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Getting the website title
        self.title = soup.title.string if soup.title else "No title found"
        
        # Removing irrelevant elements like scripts, styles, images, and inputs
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        
        # Extracting the clean text from the body of the page
        self.text = soup.body.get_text(separator="\n", strip=True)



In [26]:
# Let's try one out. Change the website and add print statements to follow along.
ed = Website("https://python.org/")

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [27]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [28]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [29]:
print(user_prompt_for(ed))

You are looking at a website titled Welcome to Python.org
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Notice:
While JavaScript is not essential for this website, your interaction with the content will be limited. Please turn JavaScript on for the full experience.
Skip to content
▼
Close
Python
PSF
Docs
PyPI
Jobs
Community
▲
The Python Network
Donate
≡
Menu
Search This Site
GO
A
A
Smaller
Larger
Reset
Socialize
LinkedIn
Mastodon
Chat on IRC
Twitter
About
Applications
Quotes
Getting Started
Help
Python Brochure
Downloads
All releases
Source code
Windows
macOS
Other Platforms
License
Alternative Implementations
Documentation
Docs
Audio/Visual Talks
Beginner's Guide
Developer's Guide
FAQ
Non-English Docs
PEP Index
Python Books
Python Essays
Community
Diversity
Mailing Lists
IRC
Forums
PSF Annual Impact Report
Python Conferences
Special Interest Groups
Python Logo
Pyt

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]

To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [30]:
from ollama import chat

messages = [
    {"role": "system", "content": "You are a medical man who knows most of medical term and also the best part is you are a doctor"},
    {"role": "user", "content": "i have a throat isuue"}
]

In [31]:
# To give you a preview -- calling OpenAI with system and user messages:

response = chat(model="deepseek-r1:1.5b", messages=messages)
print(response['message']['content'])

<think>
Okay, so I just had this serious throat issue that's not going away. I'm trying to figure out what's going on and how to treat it. Let me start by recalling some of the information I know.

First, the user mentioned they've been having trouble with their throat for a while now. They said the pain started suddenly and has been worse since then. That makes me think about different types of throat infections or conditions. I remember hearing terms like acid causing throat problems, something about mucus in the airways, and things to do when you can't see through your nose.

I think this could be an example of a viral infection because it's something that happens all year around but is specific to the body. Acute infections are those where symptoms last for a few days or weeks, which fits here since it started suddenly. Viral infections cause inflammation and swelling in tissues, which makes sense with how the throat feels.

So, my next thought is, what if this isn't something you 

## And now let's build useful messages for GPT-4o-mini, using a function

In [32]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [33]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Welcome to Python.org\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nNotice:\nWhile JavaScript is not essential for this website, your interaction with the content will be limited. Please turn JavaScript on for the full experience.\nSkip to content\n▼\nClose\nPython\nPSF\nDocs\nPyPI\nJobs\nCommunity\n▲\nThe Python Network\nDonate\n≡\nMenu\nSearch This Site\nGO\nA\nA\nSmaller\nLarger\nReset\nSocialize\nLinkedIn\nMastodon\nChat on IRC\nTwitter\nAbout\nApplications\nQuotes\nGetting Started\nHelp\nPython Brochure\nDownloads\nAll releases\nSource code\nWindows\nmacOS\nOther Platforms\nLicense\nAlternative Im

## Time to bring it together - the API for OpenAI is very simple!

In [ ]:
from ollama import chat

def summarize(url):
    website = Website(url)  # Assuming this fetches and processes the website content
    response = chat(  # Directly calling the function
        model="deepseek-r1:1.5b",
        messages=messages_for(website)
    )
    print(response)
    return response["message"]

summarize("https://edwarddonner.com")


In [39]:
summarize("https://edwarddonner.com")

Message(role='assistant', content='<think>\nAlright, so I\'m trying to summarize this website based on the content provided. Let\'s break it down step by step.\n\nFirst, I see that the website is titled "Home - Edward Donner." This immediately suggests that Edward Donner owns or operates this site and probably has various pages or sections dedicated to his work or project. The next part is under a navigation menu: Connect Four, Outsmart, an arena for LLMs, and About. These are likely different sections of the site.\n\nLooking at "About," it lists several topics like posts, job descriptions for Ed, information on AI in talent sourcing, and details about his startup called Nebula.io. This tells me that Edward isn\'t just a developer but also a business owner or entrepreneur involved with AI and talent management.\n\nThe navigation menu includes Connect Four, Outsmart, LLMs arena, and an About section. Each of these seems to be different areas within the site\'s structure. Connect Four mi

In [50]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    summary_text = summary['content'] if isinstance(summary, dict) else str(summary)
    display(Markdown(summary_text))

In [51]:
display_summary("https://edwarddonner.com")

role='assistant' content="<think>\nOkay, so I'm trying to summarize this website. Let's see... It starts with Home, then lists some categories like Connect Four, Outsmart, and an arena for LLMs. There's also a About section with posts, and navigation links. The name is Edward Donner.\n\nHmm, I think the main thing here is that it's about AI or LLM stuff. Because there are mentions of Connect Four and Outsmart games. Oh, right, those can be used as platforms for LLMs in a game setting. That's interesting. So maybe it's an AI-powered platform where users can play these connect four and outsmart games to simulate LLM interactions.\n\nAlso, the navigation has some specific links like Connect Four, Outsmart, etc., which makes sense since they're named after those games. The About section talks about writing code for LLMs and other things. They mention using tools like Python and TensorFlow, which are common in AI development. So it's a tool for developers and researchers.\n\nThe user is described as the co-founder of Nebula.io, an AI platform. So Nebula.io uses these LLMs to help people discover their potential or manage their careers. The website also has a newsletter subscription and follows various social media platforms for updates.\n\nPutting it all together, this seems like a platform designed for developers and AI enthusiasts. It provides resources, courses, and tools that can be used in the AI field. The games mentioned are just background information on how LLMs work or are being used within the application.\n\nI wonder if there's more to the website. Maybe some case studies or testimonials? Or maybe a blog section with recent articles about using these platforms for LLMs? Also, since it's an FAQ section at the end, it might have answered common questions about how to use the platform effectively.\n\nOverall, I think this is a comprehensive AI development and support site focused on tools that help in creating LLM-based applications. It seems well-structured with clear categories and useful links for developers looking to build or enhance their AI projects.\n</think>\n\nThe website Edward Donner is an AI-powered platform designed for developers and researchers interested in using Large Language Models (LLMs). Here's a structured summary:\n\n1. **Website Overview**:\n   - **Home**: A central hub offering various categories and resources related to AI development.\n   - **Categories**: Includes sections like Connect Four, Outsmart, and an LLM arena, which simulate game interactions for LLMs.\n\n2. **Content and Features**:\n   - **About**: Focuses on the development of coding tools for LLMs and their applications in fields such as career management and talent discovery.\n   - **Co-founder**: Ed. is a co-founder of Nebula.io, an AI platform that leverages these LLMs to assist users in discovering potential or managing their careers.\n\n3. **Navigation**:\n   - Provides specific links for games like Connect Four and Outsmart, suggesting the use of these as platforms for simulating LLM interactions.\n\n4. **Resources and Tools**:\n   - Offers a comprehensive directory with courses, tutorials, and tools for AI development.\n   - Includes case studies and testimonials, along with a newsletter subscription for updates on AI trends.\n\n5. **Community and Engagement**:\n   - Features a FAQ section to address common questions about using the platform effectively.\n   - Provides access to social media platforms for community engagement and insights.\n\n6. **Conclusion**: The website serves as a hub for developers and researchers, offering resources and tools to enhance LLM-based applications in AI development and research.\n\nIn summary, Edward Donner is an essential resource for anyone looking to develop or utilize AI systems, with a focus on tools that facilitate LLM-based applications in various fields." images=None tool_calls=None

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [52]:
display_summary("https://cnn.com")

role='assistant' content='<think>\nAlright, so I need to figure out how many times the word "calculus" appears in the given text. Let\'s start by reading through the text carefully and then go step by step.\n\nFirst, I\'ll read each sentence one by one and check for any instances of the letter \'c\' that might be part of a longer word, like calculus. \n\n1. The text starts with "The famous mathematician…" – no \'calculus\' here.\n2. Next is a quote from someone mentioning something about calculus in the context of mathematics. Hmm, "I don\'t think you can get an average derivative without calculus." Okay, that\'s two words: "derivative," but "average" doesn\'t have \'c\', so only one word there.\n\nWait, no. Let me check again. The quote says, "I don’t think you can get an average derivative without calculus." So it\'s "derivative" and "calculus." But the second part is about getting a derivative, which requires calculus. So in this sentence, both words have \'c\': \'derivative\' has two letters with \'c\'? No, \'derivative\' starts with \'de\', so only one \'d\'. Wait, no: \'derivative\' has an \'e\', then \'r\', but the word doesn\'t have a \'c\'. So actually, in this sentence, it\'s "derivative" and "calculus." Only calculus has two letters starting with \'c\'. But I\'m looking for occurrences of \'c\'.\n\nWait, maybe there are more. Let me go through each part again.\n\n- "Derivative": d-e-r-i-v-a-tive – no \'c\'.\n- "Calculus": c-a-l-c-u-lus – yes, that\'s two \'c\'s in a row here.\n\nBut the sentence is: "I don’t think you can get an average derivative without calculus." So the word "derivative" comes after "average," which doesn\'t have a \'c\', but "calculus" itself has two \'c\'s. So that\'s one instance of \'c\' in a word, specifically within \'calculus\'.\n\nWait, is there another part? Let me look again.\n\n- The quote: "I don’t think you can get an average derivative without calculus." So only the word \'derivative\' and \'calculus.\' But in this sentence, \'derivative\' is right after \'average,\' which doesn\'t have a \'c\', but \'calculus\' itself has two \'c\'s. Is that considered? Or do I count each letter individually?\n\nWait, no. If I\'m just looking for the word "calculus," regardless of how it\'s spelled, then in this sentence, only the word "calculus" is present with two \'c\'s. So that\'s one occurrence.\n\nBut hold on, maybe I missed something else. Let me check other parts.\n\n- The next part: "I don’t think you can get an average derivative without calculus." That\'s it for this text.\n\nLooking further down:\n\n- "The famous mathematician… I don\'t think you can get an average derivative without calculus." So that\'s just \'calculus.\'\n\nThen, moving on to the next sentence:\n\n- "But if I didn’t like how he approached things, maybe I could have had a different opinion. I mean, there’s no reason for him to come off as…"\n  \nNo \'c\' here.\n\nNext: "However, it is clear that one shouldn\'t…"\n\nNope.\n\n- "But I don’t think you can get an average derivative without calculus." Again, same sentence as before. So only the word \'calculus\'.\n\nContinuing:\n\n- "I wonder if we can figure out anything meaningful. Maybe just for…"\n  \nNo \'c\'s.\n\n- "Wait a minute, is this all you need? I mean, maybe I should have known…"\n\nNo \'c\'s.\n\nNext sentence: "But even though he may have been a… 1968."\n\nNo \'c\'s.\n\nThen:\n\n- "Maybe I can’t just give up on anything…"\n\nNope.\n\nNext part about the movie: "Ryan Gosling set to embark on journey to ‘a galaxy far, far away’ in a new ‘Star Wars’ movie." No \'c\'s here.\n\nLooking further down:\n\n- "The Assignment with Audie Cornish" – nope.\n\n- "One Thing" – no \'c\'s.\n\nWait, but earlier I thought of two \'c\'s in the word "derivative," but since it doesn\'t start with a \'c,\' only the calculus word counts. So now that I think about it again, maybe in some other part?\n\nLooking back to the initial sentences: \n\n- The first sentence is "The famous mathematician… without calculus." Wait, no, it\'s "without calculus" – yes, so here we have two \'c\'s in a row.\n\nSo let me clarify:\n\nWhen I read this sentence: "I don’t think you can get an average derivative without calculus."\n\nBreaking it down:\n\n- "I don’t think you can get an average derivative." Here, the word is "derivative," which starts with \'de,\' so only one \'d.\' The word "without" has an \'o\' and \'u,\' no \'c\'s.\n\nBut then, the phrase "without calculus" – that\'s two \'c\'s in a row.\n\nSo that\'s another instance of \'calculus\' appearing twice. So that\'s one occurrence.\n\nWait, but is it considered when we just have the word \'derivative\'? Or do I count each \'c\' individually?\n\nThe question is asking how many times "calculus" appears. It doesn\'t specify case sensitivity or if it should be counted multiple times within a single word or only as separate letters. So probably, I need to consider both \'c\'s in the phrase "without calculus."\n\nSo that would mean one occurrence of \'c\'s within the \'calculus\' part.\n\nWait, but actually, let me check:\n\n- The word is: "without calculus." So the first \'c\' comes after \'out,\' and then the second \'c\' comes after \'without.\'\n\nSo in this sentence, we have two instances where \'c\' appears: one in \'without\' (as the \'o\'), but wait, no. \'Without\' starts with \'un-\'; the word is spelled \'u-n-h-i-o.\' So only one \'c\' before that?\n\nWait, perhaps not. Let me spell it out:\n\n- "without calculus"\n\nLetters: u-n-h-i-o-u-n-c-a-l-u-l-s\n\nSo within this phrase, \'calculus\' starts at position where? Wait, no.\n\nActually, the word is spelled as \'u-n-h-i-o-U-N-C-A-L-U-L-S.\'\n\nWait, that\'s confusing. The correct spelling of calculus is c-a-l-u-t-e-r?\n\nNo, wait, actually, it\'s c-a-l-c-u-l-u-s.\n\nSo in "without calculus," the \'c\' comes after \'out,\' then another \'c\' when we get to \'without.\' So two \'c\'s in a row: first \'c\' before \'out,\' and second \'c\' when we reach \'without.\'\n\nBut actually, let me spell it out correctly.\n\nThe correct spelling is C-A-L-C-U-L-U-S. So in the sentence "without calculus," the word starts with \'u-n-h-i-o-u-n-c-a-l-u-l-s.\' So within that phrase, the first \'c\' comes after \'o\' when it\'s spelled as \'u-n-h-i-o-U-N-C...\' Wait, maybe I\'m overcomplicating.\n\nIn English, "without calculus" is two separate words: without and calculus. So in this sentence, we have a space between them, meaning two instances of \'c\'s. The first \'c\' is part of the word "without," but wait, no. \'Without\' doesn\'t end with a \'c.\' Wait, that\'s incorrect.\n\nWait, actually, in the phrase "without calculus," the structure is: without + calculus. So within this combined phrase, we have two separate words: without and calculus. The first \'c\' in "without" comes after \'u-n-h-i-o-U-N-C...\' Hmm, maybe not. Let me count letter by letter.\n\nWait, this is getting confusing. Maybe I should go through the entire text again and look for all instances of \'calculus.\'\n\nLooking at each sentence:\n\n1. The famous mathematician… without calculus. – Here, \'without\' has an \'o\' before a capital \'u,\' so no \'c\'s in there.\n\nWait, actually, \'without calculus\' is two separate words: "without" and "calculus." So within that phrase, how many times does \'c\' appear? It\'s one \'c\' when we reach \'c-a-l-c-u-l-u-s.\' Wait, no. Let me write it out:\n\nIn "without calculus," the letters are u-n-h-i-o-U-N-C-A-L-U-L-S.\n\nSo within that phrase, the word is spelled as follows: u-n-h-i-o-U-N-C-A-L-U-L-S.\n\nWait, actually, when you spell \'without calculus,\' it\'s two separate words: without and calculus. So in "without," there\'s an \'o\' before a capital \'u,\' so no \'c\'s. But within the word \'calculus,\' which is c-a-l-c-u-l-u-s, how many \'c\'s are present? It has two \'c\'s.\n\nSo, if I\'m just counting how many times \'c\' appears in any part of the text, including within a single word or across words, then this would be one occurrence of \'calculus.\'\n\nBut wait, no. Because within "without calculus," there is a single \'c\' when you spell it out as c-a-l-c-u-l-u-s. So that\'s another \'c.\' But actually, in the phrase "without calculus," we have two separate words: without and calculus. The first word doesn\'t contain any \'c\'s, but within the second word, \'calculus,\' there are two \'c\'s.\n\nBut wait, hold on. Is it considered when you count letters across words? If I\'m just counting the total number of \'c\'s in the entire sentence, regardless of where they are, then in "without calculus," we have two \'c\'s: one at position 4 (if starting from 0) and another maybe?\n\nWait, let me write it out:\n\nThe phrase is "without calculus."\n\nBreaking it down letter by letter:\n1. u\n2. n\n3. h\n4. i\n5. o\n6. U\n7. N\n8. C\n9. A\n10. L\n11. U\n12. L\n13. U\n14. S\n\nSo in this phrase, the \'C\' is at position 8. So only one \'c.\'\n\nBut wait, that\'s not correct because when you spell \'without calculus,\' it\'s two words: without and calculus.\n\nWithout has an \'o\' before a capital \'u.\' Calculus is spelled c-a-l-c-u-l-u-s.\n\nSo the letter sequence for "without" would be u-n-h-i-o-U-N-C-A-L-U-L-S. Wait, that doesn\'t make sense because in English, when you have "without," it\'s u-n-h-i-o-U-N-C-A-L-U-L-S.\n\nWait, perhaps I\'m overcomplicating this. Maybe the better approach is to count all \'c\'s regardless of case and across words. So if we just look at the entire text:\n\nLooking for every occurrence of \'c\' in any word or phrase, whether it\'s part of a separate word or within one.\n\nSo let me scan each sentence again.\n\n1. The famous mathematician… without calculus.\n   - No \'c\'s here except maybe in some words? No, I don\'t think so.\n\n2. If you give them the gift of knowledge, they will learn the knowledge of life… calculus.\n   - Here, it says "without calculus." So that has a single \'c.\'\n\n3. The key to success is to stay focused on your goal and never allow anyone to interfere with your plans. Calculus is one of the first subjects you need to master in this world… mathematics.\n   - No \'c\'s here.\n\n4. You can’t be perfect without proper preparation. And calculus isn’t something that gets everyone’s attention when they’re tired. So you have to take it easy, not study for too long before bed.\n   - No \'c\'s here.\n\n5. Don’t try to force someone else to do the work; just let them do it on their own and see how well they can figure things out without calculus.\n   - No \'c\'s.\n\n6. The key to success is to stay focused on your goal and never allow anyone to interfere with your plans. Calculus is one of the first subjects you need to master in this world… mathematics.\n   - No \'c\'s here.\n\n7. Don’t overthink it, don\'t be too hard on yourself when things go south. Because calculus isn’t going anywhere without proper preparation.\n   - No \'c\'s.\n\n8. The key to success is to stay focused on your goal and never allow anyone to interfere with your plans. Calculus is one of the first subjects you need to master in this world… mathematics.\n   - No \'c\'s here.\n\n9. Don’t let them go to your head, don\'t force them to do the work for you; just let them do it on their own and see how well they can figure things out without calculus.\n   - No \'c\'s.\n\n10. You know, maybe I should have tried studying a bit more today, before bed. And calculus isn’t something that gets everyone’s attention when they’re tired. So you have to take it easy, not study for too long before bed.\n    - No \'c\'s here.\n\nSo in the entire text, how many times does "calculus" appear? Let me see:\n\n- First occurrence: In sentence 2, within "without calculus," we have one \'c\' at position 8 (if counting from 0). So that\'s one \'c.\'\n\nIs there any other \'c\' elsewhere?\n\nLooking through the text, I don\'t see any other \'c\'s. So that would be just one occurrence of "calculus."\n\nWait, but earlier in sentence 2: "without calculus." Is the \'C\' uppercase or lowercase? In English, it\'s usually written with capital letters for words, but sometimes lowercase if it\'s a proper noun.\n\nIn this context, since we\'re talking about a class and calculus is just an academic subject, I think it would be better to write it as "without calculus" with the first letter uppercase. So maybe in sentence 2: "without Calculus." That way, there\'s only one \'C\' at position 8.\n\nSo that would make it one occurrence of "Calculus," but if we\'re counting both cases, then it might be two? But generally, in academic contexts, calculus is lowercase unless it\'s a proper noun or an acronym. So it should probably be written as "without Calculus." That way, only one \'C\' appears.\n\nTherefore, in the entire text, the word "Calculus" occurs once.\n</think>\n\nThe word "calculus" occurs once in the given text. It is written with the capital letter at position 8, making it part of the phrase "without Calculus."\n\n**Answer:** The calculus word occurs once.' images=None tool_calls=None

In [ ]:
display_summary("https://anthropic.com")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise, you experienced calling the Cloud API of a Frontier Model (a leading model at the frontier of AI) for the first time. We will be using APIs like OpenAI at many stages in the course, in addition to building our own LLMs.

More specifically, we've applied this to Summarization - a classic Gen AI use case to make a summary. This can be applied to any business vertical - summarizing the news, summarizing financial performance, summarizing a resume in a cover letter - the applications are limitless. Consider how you could apply Summarization in your business, and try prototyping a solution.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue - now try yourself</h2>
            <span style="color:#900;">Use the cell below to make your own simple commercial example. Stick with the summarization use case for now. Here's an idea: write something that will take the contents of an email, and will suggest an appropriate short subject line for the email. That's the kind of feature that might be built into a commercial email tool.</span>
        </td>
    </tr>
</table>

In [59]:
# Step 1: Create your prompts

system_prompt = """you are a top class and a friendly doctor and whenever the users ask you about a medical issue like for example i have a fever now if user 
provides only one condition so now you have to ask a follow-up question about the users condition based on the answer of your follow-up question provide a prescrition to user in a list format and how to and when to take that particular medicine"""
user_prompt = """
    I’ve been experiencing persistent fatigue for the past few weeks, which seems to worsen throughout the day, even after getting a full night’s sleep. In addition, I've had occasional discomfort in my chest, which feels like mild pressure or tightness
"""

# Step 2: Make the messages list



messages = [
    {"role":"system","content":system_prompt},
    {"role":"user","content":user_prompt},
] # fill this in

# Step 3: Call OpenAI


response = chat(model="deepseek-r1:1.5b",messages=messages)

# Step 4: print the result
response_content = response["message"]["content"]
print(response_content)

<think>
Alright, let's tackle this user. They mentioned feeling fatigued all day and some chest discomfort after sleep. That points towards fatigue.

I should start by addressing the cause. Could it be something like sleep apnea? That seems likely given the symptoms, especially the fatigue.

So, first question: Have they had any sleep issues before now? If not, then aseptic determination would be good to rule out other causes.

If they do have sleep apnea, next step is to determine if it's obstructive. They might need an assessment to confirm that.

After ruling out obstruction or lack of sleep, they should start with oral decongestants to manage the chest pressure. Then, if symptoms improve, switch to a nasal nasal spray for more relief.

If both options don't work, they'll have to consider alternatives like oral antihistamines, but these might be tricky. They also need to consult their doctor soon because their symptoms could get worse without proper management.

I should present thi

## An extra exercise for those who enjoy web scraping

You may notice that if you try `display_summary("https://openai.com")` - it doesn't work! That's because OpenAI has a fancy website that uses Javascript. There are many ways around this that some of you might be familiar with. For example, Selenium is a hugely popular framework that runs a browser behind the scenes, renders the page, and allows you to query it. If you have experience with Selenium, Playwright or similar, then feel free to improve the Website class to use them. In the community-contributions folder, you'll find an example Selenium solution from a student (thank you!)

# Sharing your code

I'd love it if you share your code afterwards so I can share it with others! You'll notice that some students have already made changes (including a Selenium implementation) which you will find in the community-contributions folder. If you'd like add your changes to that folder, submit a Pull Request with your new versions in that folder and I'll merge your changes.

If you're not an expert with git (and I am not!) then GPT has given some nice instructions on how to submit a Pull Request. It's a bit of an involved process, but once you've done it once it's pretty clear. As a pro-tip: it's best if you clear the outputs of your Jupyter notebooks (Edit >> Clean outputs of all cells, and then Save) for clean notebooks.

Here are good instructions courtesy of an AI friend:  
https://chatgpt.com/share/677a9cb5-c64c-8012-99e0-e06e88afd293